# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Creating a keyspace for Cassandra
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Setting the current session's keyspace to the one that we've created
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
"""
    Query Description: In this query, I decided to use the session_id as the partition key to make the partition unique. item_in_session column
    is used as a clustering column, forming a composite primary key with the session_id so that each row is uniquely identifiable because
    there is no duplicate item_in_session relating to a specific session_id.
"""
query_one = "CREATE TABLE IF NOT EXISTS song_library "
query_one += "(session_id int, item_in_session int, artist_name text, song_name text, length float, PRIMARY KEY(session_id, item_in_session))"

try:
    rows = session.execute(query_one)
except Exception as e:
    print(e)
    


                    

In [10]:
# Open the .csv data file, extracts the data and inserting them into the song_library table 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_library (session_id, item_in_session, artist_name, song_name, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)


#### Do a SELECT to verify that the data have been inserted into each table

In [11]:

select_one = "SELECT artist_name AS artist, song_name AS song_title, length AS song_length FROM song_library WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(select_one)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
"""
    Query Description: In this query, I decided to use the user_id as the partition key to make the partition unique. 
    session_id and item_in_session columns are used as a clustering columns. session_id column is used as a clustering
    column to make each row of data distinct and item_in_session is also used because we want to sort the songs based on
    it.
"""
query_two = "CREATE TABLE IF NOT EXISTS user_logs "
query_two += "(user_id int, session_id int, item_in_session int, first_name text, last_name text, artist_name text, song_name text, PRIMARY KEY(user_id, session_id, item_in_session))"

try:
    session.execute(query_two)
except Exception as e:
    print(e)

                    

In [13]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_logs (user_id, session_id, item_in_session, first_name, last_name, artist_name, song_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))
        except Exception as e:
            print(e)


In [14]:
select_two = "SELECT artist_name AS artist, song_name AS song, first_name, last_name, item_in_session FROM user_logs WHERE  user_id = 10 AND session_id = 182"
try:
    rows = session.execute(select_two)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.item_in_session, row.artist, row.song, f"{row.first_name} {row.last_name}")

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
"""
    Query Description: In this query, I decided to use the song_name as the partition key to make the partition unique and
    to be able to query using it. user_id is used as a clustering columns to make each row of data uniquely identifiable and
    only it is used as a clustering column because I think it is enough for the query.
"""
query_three = "CREATE TABLE IF NOT EXISTS songplay_history "
query_three += "(song_name text, user_id int, first_name text, last_name text, PRIMARY KEY(song_name, user_id))"
session.execute(query_three)

                    
                  

In [16]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO songplay_history (song_name, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)


In [28]:
select_three = "SELECT first_name, last_name FROM songplay_history WHERE song_name ='All Hands Against His Own'"
try:
    rows = session.execute(select_three)
except Exception as e:
    print(e)
    
for row in rows:
    print (f"{row.first_name} {row.last_name}")


Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [29]:
# Creating a list that contains the name of the tables
tables = ['song_library', 'user_logs', 'songplay_history']

# Loop and drop all tables that had been created
for table in tables:
    try:
        rows = session.execute(f"DROP TABLE {table}")
    except Exception as e:
        print(e)



### Close the session and cluster connection¶

In [91]:
session.shutdown()
cluster.shutdown()